In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import os
from math import log
import numpy as np
import scipy as sp
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
pe_data_path = "/content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign"
bytes_data_path = "/content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_byte"
img_data_path = "/content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/Benign"
csv_data_path = "/content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/data.csv"

In [ ]:

def pe2hex(file_path, output_file_path):
  print('Processing '+file_path)
  file = bytearray(open(file_path, 'rb').read())
  key = "\0"
  with open(output_file_path, 'w') as output:
      for count, byte in enumerate(file, 1):
          output.write(
              f'{byte ^ ord(key[(count - 1) % len(key)]):#0{4}x}' + (
                  '\n' if not count % 16 else ' '))


In [ ]:
def hex2img(array, output_img_path):
    if array.shape[1]!=16: #If not hexadecimal
        assert(False)
    b=int((array.shape[0]*16)**(0.5))
    b=2**(int(log(b)/log(2))+1)
    a=int(array.shape[0]*16/b)
    print(a,b,array.shape)
    array=array[:a*b//16,:]
    array=np.reshape(array,(a,b))
    im = Image.fromarray(np.uint8(array))
    #out = im.transpose(Image.FLIP_LEFT_RIGHT )
    im.save(output_img_path, "PNG")
    return im





**Convert PE files to binary code**






In [ ]:
files= os.listdir(pe_data_path)

for counter, name in enumerate(files):
  name_output = name.split(".")[0]
  print(name_output)
  pe2hex(os.path.join(pe_data_path,name), os.path.join(bytes_data_path,name_output+".bytes"))

1fcbe5d51360bee55f55c791a95df8c61f73e117bda89b9439d74df5cee7aa71
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign/1fcbe5d51360bee55f55c791a95df8c61f73e117bda89b9439d74df5cee7aa71.exe
605277968dd1b2f324f1563f0c67a7225715c3d2859c6a435e0c5ef519093621
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign/605277968dd1b2f324f1563f0c67a7225715c3d2859c6a435e0c5ef519093621.ole
2f031a1752f7067fb9f483ae0ac5f3036c9b66cc4af40e6ae609c0aba7c886d0
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign/2f031a1752f7067fb9f483ae0ac5f3036c9b66cc4af40e6ae609c0aba7c886d0.exe
311129b9debdb192020e2038697de1f23bc8572d02dfbfeb742b3f603df950da
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign/311129b9debdb192020e2038697de1f23bc8572d02dfbfeb742b3f603df950da.exe
282f6f7fb0c5b0de73b72301646791901ae28c8c69cc25cf47b2ff773cb5dc20
Processing /content/drive/MyDrive/content/drive/MyD

**Convert binary code to Inmage**

In [ ]:
df_benign = pd.read_csv(csv_data_path, index_col=False)
already_transformed = list(df_benign['img_code'])
benign_class_index = 25

In [ ]:
files= os.listdir(bytes_data_path)

for counter, name in enumerate(files):
        name_output = name.split(".")[0]

        output_image_path = os.path.join(img_data_path,name_output+".png")
        relative_image_path = os.path.join("./benign_data/benign_imgs/Benign", name_output+".png")

        if(relative_image_path in already_transformed):
         continue

        print('Processing '+output_image_path)

        f=open(os.path.join(bytes_data_path,name), 'r')

        array=[]
        for line in f:
            xx=line.replace("\n", "").split(" ")

            if(len(xx) != 16 or "" in xx):
              continue

            array.append([int(i,16) if i!='??' else 0 for i in xx])

        img = hex2img(np.array(array),output_image_path)
        del array

        df_benign.loc[len(df_benign.index)] = [relative_image_path, benign_class_index]

        f.close()

Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/Benign/1fcbe5d51360bee55f55c791a95df8c61f73e117bda89b9439d74df5cee7aa71.png
353 512 (11296, 16)
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/Benign/605277968dd1b2f324f1563f0c67a7225715c3d2859c6a435e0c5ef519093621.png
202 256 (3232, 16)
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/Benign/2f031a1752f7067fb9f483ae0ac5f3036c9b66cc4af40e6ae609c0aba7c886d0.png
138 256 (2208, 16)
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/Benign/311129b9debdb192020e2038697de1f23bc8572d02dfbfeb742b3f603df950da.png
408 512 (13056, 16)
Processing /content/drive/MyDrive/content/drive/MyDrive/progetto_SS/benign_data/benign_imgs/Benign/282f6f7fb0c5b0de73b72301646791901ae28c8c69cc25cf47b2ff773cb5dc20.png
279 512 (8928, 16)
Processing /content/drive/MyDrive/content/drive/MyDrive/progett

In [ ]:
df_benign.to_csv(csv_data_path, index=False)